In [ ]:
import cv2
import time
from collections import deque

# Initialize cameras
cap1 = cv2.VideoCapture(0)  # Laptop webcam
cap2 = cv2.VideoCapture(1)  # USB camera

# Load object detection model
# You'll need to download a pre-trained model and configure the paths accordingly
model_path = 'path/to/frozen_inference_graph.pb'
config_path = 'path/to/pipeline.config'
detector = cv2.dnn.readNetFromTensorflow(model_path, config_path)

# Initialize multi-object tracker
tracker = cv2.MultiTracker_create()

# Set up windows for dual-camera display
cv2.namedWindow('Camera 1', cv2.WINDOW_NORMAL)
cv2.namedWindow('Camera 2', cv2.WINDOW_NORMAL)

# Track objects across both cameras
tracked_objects = deque()

while True:
    # Capture frames from both cameras
    ret1, frame1 = cap1.read()
    ret2, frame2 = cap2.read()

    if ret1 and ret2:
        # Detect people in each frame
        boxes1, scores1, classes1, nums1 = detector.forward(frame1)
        boxes2, scores2, classes2, nums2 = detector.forward(frame2)

        # Update and draw tracked objects
        success, boxes1 = tracker.update(frame1)
        success, boxes2 = tracker.update(frame2)
        for i, (x, y, w, h) in enumerate(boxes1):
            cv2.rectangle(frame1, (int(x), int(y)), (int(x + w), int(y + h)), (0, 255, 0), 2)
        for i, (x, y, w, h) in enumerate(boxes2):
            cv2.rectangle(frame2, (int(x), int(y)), (int(x + w), int(y + h)), (0, 255, 0), 2)

        # Display FPS
        fps1 = 1.0 / (time.time() - start_time1)
        fps2 = 1.0 / (time.time() - start_time2)
        cv2.putText(frame1, f'FPS: {fps1:.2f}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
        cv2.putText(frame2, f'FPS: {fps2:.2f}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

        # Display frames
        cv2.imshow('Camera 1', frame1)
        cv2.imshow('Camera 2', frame2)

        # Update start times for FPS calculation
        start_time1 = time.time()
        start_time2 = time.time()

    # Check for user input to exit
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

# Release resources and close windows
cap1.release()
cap2.release()
cv2.destroyAllWindows()